<a href="https://colab.research.google.com/github/mario1870/swabianGPT/blob/main/swabianGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Finetuning eines schwäbischen LLMs mit LoRA**##

# Schritt 1 - Einrichtung

**Notwendige Abhängigkeiten installieren**<br>
-q: "quiet" - zeigt nur Warnungen und Fehler<br>
-U: "upgrade" - aktualisiert das Paket auf die neueste Version

In [1]:
%%capture
%pip install -q -U transformers
%pip install -q -U datasets
%pip install -q -U accelerate
%pip install -q -U peft
%pip install -q -U trl
%pip install -q -U bitsandbytes
%pip install -q -U wandb

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch
from trl import SFTTrainer, setup_chat_format
import pandas as pd
from datasets import Dataset
import random
from huggingface_hub import login

ModuleNotFoundError: No module named 'trl'

Login mit Huggingface zum laden des Modells

In [3]:
from google.colab import userdata

# Token aus Colab Secrets laden
token = userdata.get('hf_token')
login(token=token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Initialisierung von W&B zum Tracken verschiedener Metriken

In [ ]:
import wandb

wb_token = userdata.get("wanb")

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune Llama 3.2 on Customer Support Dataset',
    job_type="training",
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: marioraach01 (marioraach01-student). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Schritt 2 - Modell & Tokenizer laden

In [ ]:
base_model = "meta-llama/Llama-3.2-3B-Instruct"
new_model = "llama-3.2-3b-it-swabian"
dataset_name = "bitext/Bitext-customer-support-llm-chatbot-training-dataset"

In [ ]:
bnb_config = BitsAndBytesConfig(
    # 4-bit Quantisierung aktivieren
    load_in_4bit=True,

    # Typ der 4-bit Quantisierung
    bnb_4bit_quant_type="nf4",  # Normal Float 4 - bessere Genauigkeit als int4

    # Berechnungstyp für die Quantisierung
    bnb_4bit_compute_dtype=torch.float16,

    # Doppelte Quantisierung für zusätzliche Speicherersparnis
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_type="fp16",
    trust_remote_code=True
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    # Siehe obere Config
    quantization_config=bnb_config,
    device_map="auto",

    #
    attn_implementation="eager"
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

# Optional: Daten vorverarbeiten

In [ ]:
def get_templates():
    """Definiert und gibt die Templates für die Übersetzungen zurück."""
    return {
        'schwaebisch_to_deutsch': [
            "Übersetze folgenden schwäbischen Text ins Hochdeutsche:\n{input}\nHochdeutsche Übersetzung: {output}",
            "Schwäbischer Text:\n{input}\nÜbersetze ins Hochdeutsche: {output}",
            "Was bedeutet der schwäbische Ausdruck {input} auf Hochdeutsch?\nBedeutung: {output}",
            "Kannst du mir erklären, was '{input}' auf Hochdeutsch bedeutet?\nErklärung: {output}",
            "Ich habe den schwäbischen Ausdruck '{input}' gehört. Was heißt das?\nBedeutung: {output}",
            "'{input}' hat jemand zu mir gesagt. Wie würde man das auf Hochdeutsch sagen?\nHochdeutsch: {output}",
            "Schwäbisch: {input}\nKannst du das ins Hochdeutsche übersetzen?\nÜbersetzung: {output}",
            "Wie würde man '{input}' in normalem Deutsch ausdrücken?\nDeutsche Version: {output}",
            "Hilf mir bitte diesen schwäbischen Ausdruck zu verstehen: {input}\nBedeutung: {output}"
        ],
        'deutsch_to_schwaebisch': [
            "Übersetze folgenden deutschen Text ins Schwäbische:\n{input}\nSchwäbische Übersetzung: {output}",
            "Deutscher Text:\n{input}\nÜbersetze ins Schwäbische: {output}",
            "Wie sagt man {input} auf Schwäbisch?\nSchwäbisch: {output}",
            "Wie würde ein Schwabe '{input}' sagen?\nSchwäbische Version: {output}",
            "Ich möchte '{input}' auf Schwäbisch sagen. Wie geht das?\nSchwäbisch: {output}",
            "Kannst du mir beibringen, wie man '{input}' auf Schwäbisch sagt?\nAuf Schwäbisch: {output}",
            "In Stuttgart möchte ich '{input}' sagen. Wie sagt man das dort?\nStuttgarter Version: {output}",
            "Hilf mir '{input}' ins Schwäbische zu übersetzen.\nSchwäbische Übersetzung: {output}",
            "Auf Schwäbisch würde man für '{input}' was sagen?\nAntwort: {output}"
        ]
    }


In [ ]:
def split_translation(text):
    """Trennt wörtliche Übersetzungen und Erklärungen"""
    parts = {}

    # Wörtliche Übersetzung extrahieren
    if " - wörtl.: " in text:
        main, literal = text.split(" - wörtl.: ")
        parts['main'] = main.strip()
        parts['literal'] = literal.strip()
    else:
        parts['main'] = text.strip()
        parts['literal'] = None

    return parts

In [ ]:
def get_complexity(text):
    """Bestimmt Komplexität des Texts"""
    words = text.split()
    if len(words) < 3:
        return "einfach"    # Einfache Wortübersetzungen
    elif len(words) < 8:
        return "mittel"     # Kurze Sätze/Ausdrücke
    return "komplex"        # Sprichwörter/Lange Sätze

In [ ]:
def determine_category(text):
    """Kategorisiert den Text"""
    if "?" in text:
        return "frage"
    elif "!" in text:
        return "ausruf"
    elif len(text.split()) <= 2:
        return "wort"
    elif "man" in text.lower() or "würde" in text.lower():
        return "redensart"
    return "satz"

In [ ]:
def preprocess_data(file_path, column_mapping=None):
    """Vorverarbeitung der Daten aus einer CSV-Datei."""
    if column_mapping is None:
        # Anpassung an die tatsächlichen Spaltennamen aus deiner CSV
        column_mapping = {
            'A aldbachas Gloid': 'dialekt',
            'Ein aus der Mode gekommenes Kleid - wörtl.: ein altbackenes Kleidungsstück': 'hochdeutsch'
        }

    try:
        # CSV laden
        df = pd.read_csv(file_path)
        print("Originale Spalten:", df.columns.tolist())

        # Spalten umbenennen
        df = df.rename(columns=column_mapping)
        print("Neue Spalten:", df.columns.tolist())

        # NaN-Werte durch leere Strings ersetzen
        df = df.fillna('')

        # Verbesserte Datensäuberung mit Typ-Check
        for col in ['dialekt', 'hochdeutsch']:
            df[col] = df[col].astype(str)
            df[col] = df[col].str.strip()
            df[col] = df[col].apply(lambda x: ' '.join(x.split()))

        # Leere Einträge entfernen
        df = df[df['dialekt'] != '']
        df = df[df['hochdeutsch'] != '']
        df = df.drop_duplicates()

        print(f"Daten geladen: {len(df)} Einträge nach Vorverarbeitung")
        print("\nBeispieldaten:")
        print(df.head())

        return df

    except Exception as e:
        print(f"Fehler bei der Datenvorverarbeitung: {e}")
        import traceback
        traceback.print_exc()
        return None

In [ ]:
def prepare_translation_data(df):
    """Erweiterte Datenvorbereitung mit Metadaten"""
    if df is None or df.empty:
        raise ValueError("DataFrame ist leer oder None")

    training_data = []
    templates = get_templates()

    for _, row in df.iterrows():
        # Verarbeite Hochdeutsch
        deutsch = split_translation(row['hochdeutsch'])

        # Erstelle erweiterte Einträge für beide Richtungen
        for direction in ['schwaebisch_to_deutsch', 'deutsch_to_schwaebisch']:
            template = random.choice(templates[direction])

            if direction == 'schwaebisch_to_deutsch':
                input_text = row['dialekt']
                output_text = deutsch['main']
            else:
                input_text = deutsch['main']
                output_text = row['dialekt']

            entry = {
                'text': template.format(input=input_text, output=output_text),
                'type': direction,
                'metadata': {
                    'complexity': get_complexity(input_text),
                    'category': determine_category(input_text),
                    'has_literal': deutsch['literal'] is not None,
                    'literal': deutsch['literal'],
                    'length': len(input_text.split())
                }
            }
            training_data.append(entry)

    # Daten mischen und splitten
    random.shuffle(training_data)
    dataset = Dataset.from_list(training_data)
    dataset = dataset.train_test_split(test_size=0.1)

    return dataset

In [20]:
def preprocess_dataset():
    """Hauptfunktion mit zusätzlichen Statistiken"""
    df = preprocess_data('/content/drive/MyDrive/datasets/schwaebisch-lexikon.csv')
    dataset = prepare_translation_data(df)

    # Erweiterte Statistiken
    print(f"\nDataset Statistiken:")
    print(f"Trainingsdaten: {len(dataset['train'])} Beispiele")
    print(f"Testdaten: {len(dataset['test'])} Beispiele")

    # Analysen der Metadaten
    categories = {}
    complexities = {}
    for split in ['train', 'test']:
        for example in dataset[split]:
            cat = example['metadata']['category']
            comp = example['metadata']['complexity']
            categories[cat] = categories.get(cat, 0) + 1
            complexities[comp] = complexities.get(comp, 0) + 1

    print("\nKategorien-Verteilung:")
    for cat, count in categories.items():
        print(f"{cat}: {count}")

    print("\nKomplexitäts-Verteilung:")
    for comp, count in complexities.items():
        print(f"{comp}: {count}")

    # Beispiele mit Metadaten
    print("\nBeispiele aus dem Dataset:")
    for i in range(3):
        example = dataset['train'][i]
        print(f"\nBeispiel {i+1}:")
        print(f"Text: {example['text']}")
        print(f"Metadaten: {example['metadata']}")

    dataset.push_to_hub("Mario12355/schwaebisch_translations", private=True)
    return dataset

Originale Spalten: ['A aldbachas Gloid', 'Ein aus der Mode gekommenes Kleid - wörtl.: ein altbackenes Kleidungsstück']
Neue Spalten: ['dialekt', 'hochdeutsch']
Daten geladen: 12172 Einträge nach Vorverarbeitung

Beispieldaten:
                                             dialekt  \
0  A alde Kuah vrgissd gern, daß se au amol a Kal...   
1  A alde Sonndagshos am Werdag hebd nedd so lang...   
2              A Auswahl, wia d’ Mäus en de Huddzla.   
3                               A baar Schridd laufa   
4                                     A bees Briahle   

                                         hochdeutsch  
0  Eine alte Kuh vergisst gern, daß sie auch mal ...  
1  Eine alte Sonntagshose hält am Werktag nicht s...  
2  Ein großes (Waren)-Angebot - wörtl.: Eine Ausw...  
3                            Ein paar Schritte gehen  
4              Eine ungenießbare Flüssigkeit (Brühe)  

Dataset Statistiken:
Trainingsdaten: 21909 Beispiele
Testdaten: 2435 Beispiele

Kategorien-Verteilung:
wo

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

# Schritt 3 - Datenset laden & verarbeiten

Datenset aus Drive laden und aufbereiten

In [22]:
from datasets import load_dataset
import random

# Dataset von Hugging Face laden
dataset = load_dataset("Mario12355/schwaebisch_translations")

# Train/Test aufgeteilt verwenden
train_dataset = dataset["train"]
test_dataset = dataset["test"]

# Basis-Statistiken
print(f"\nDataset Statistiken:")
print(f"Trainingsdaten: {len(train_dataset)} Beispiele")
print(f"Testdaten: {len(test_dataset)} Beispiele")

# Metadaten-Analyse
categories = {}
complexities = {}
literals = {'with': 0, 'without': 0}

for example in train_dataset:
    # Zähle Kategorien
    cat = example['metadata']['category']
    categories[cat] = categories.get(cat, 0) + 1

    # Zähle Komplexität
    comp = example['metadata']['complexity']
    complexities[comp] = complexities.get(comp, 0) + 1

    # Zähle wörtliche Übersetzungen
    if example['metadata']['has_literal']:
        literals['with'] += 1
    else:
        literals['without'] += 1

# Ausgabe der Analysen
print("\nKategorien-Verteilung:")
for cat, count in categories.items():
    percentage = (count / len(train_dataset)) * 100
    print(f"{cat}: {count} ({percentage:.1f}%)")

print("\nKomplexitäts-Verteilung:")
for comp, count in complexities.items():
    percentage = (count / len(train_dataset)) * 100
    print(f"{comp}: {count} ({percentage:.1f}%)")

print("\nWörtliche Übersetzungen:")
print(f"Mit wörtlicher Übersetzung: {literals['with']}")
print(f"Ohne wörtliche Übersetzung: {literals['without']}")

# Beispiele nach Kategorien zeigen
print("\nBeispiele nach Kategorien:")
for cat in categories.keys():
    cat_examples = [ex for ex in train_dataset if ex['metadata']['category'] == cat]
    print(f"\n{cat.upper()} Beispiele:")
    # Zeige 2 zufällige Beispiele pro Kategorie
    for example in random.sample(cat_examples, min(2, len(cat_examples))):
        print(f"\nText: {example['text']}")
        print(f"Komplexität: {example['metadata']['complexity']}")
        if example['metadata']['has_literal']:
            print(f"Wörtlich: {example['metadata']['literal']}")


Dataset Statistiken:
Trainingsdaten: 21909 Beispiele
Testdaten: 2435 Beispiele

Kategorien-Verteilung:
wort: 11169 (51.0%)
ausruf: 3706 (16.9%)
satz: 5941 (27.1%)
frage: 676 (3.1%)
redensart: 417 (1.9%)

Komplexitäts-Verteilung:
einfach: 11422 (52.1%)
komplex: 2687 (12.3%)
mittel: 7800 (35.6%)

Wörtliche Übersetzungen:
Mit wörtlicher Übersetzung: 2319
Ohne wörtliche Übersetzung: 19590

Beispiele nach Kategorien:

WORT Beispiele:

Text: Wie würde man 'Alde Schapf' in normalem Deutsch ausdrücken?
Deutsche Version: Böses Schimpfwort über eine Frau
Komplexität: einfach
Wörtlich: Ein altes Schöpfgefäß

Text: Auf Schwäbisch würde man für 'Puppe' was sagen?
Antwort: Pubb
Komplexität: einfach

AUSRUF Beispiele:

Text: Übersetze folgenden deutschen Text ins Schwäbische:
Das eilt! Das ist wichtig!
Schwäbische Übersetzung: Des duad Not!
Komplexität: mittel
Wörtlich: Das tut Not

Text: Wie würde ein Schwabe 'Mein Bauch ist übervoll!' sagen?
Schwäbische Version: Mei Ranza schbannd!
Komplexität: mi

# Schritt 4 - Modell aufsetzen

Findet alle Linearen Layer im Model automatisch

In [ ]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    """
    Findet automatisch alle Linear Layer Names für LoRA-Training.
    Ausgenommen ist der lm_head Layer.

    Args:
        model: Das zu untersuchende Modell

    Returns:
        list: Namen aller Linear Layer, die für LoRA geeignet sind
    """
    cls = bnb.nn.Linear4bit  # Sucht nach 4-bit quantisierten Layern
    lora_module_names = set()

    # Durchsucht alle Module im Modell
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            # Nimmt den ersten Namen bei einzelnen Namen, sonst den letzten
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    # Entfernt lm_head (wichtig für 16-bit Stabilität)
    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')

    return list(lora_module_names)

# Alle linearen Module suchen
modules = find_all_linear_names(model)

In [4]:
# LoRA config
peft_config = LoraConfig(
    r=16,                    # Erhöht für bessere Modellkapazität
    lora_alpha=32,         # 2x r für gutes Scaling
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=False,
)

# Check if the tokenizer already has a chat template
if tokenizer.chat_template is not None:
    # If it does, set it to None to overwrite it
    tokenizer.chat_template = None

model, tokenizer = setup_chat_format(model, tokenizer)
model = get_peft_model(model, peft_config)

NameError: name 'find_all_linear_names' is not defined

In [ ]:
# LoRA Config für H100
peft_config = LoraConfig(
    r=32,                    # Größerer Rang möglich
    lora_alpha=64,          # 2 * r
    lora_dropout=0.05,
    target_modules=modules,
    bias="none",
    task_type="CAUSAL_LM"
)

# Training Arguments für H100
training_arguments = TrainingArguments(
    output_dir="./schwäbisch_model",
    num_train_epochs=5,

    # H100-optimierte Batch Größen
    per_device_train_batch_size=16,   # Doppelte Batch Size
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,    # Nicht nötig wegen größerer Batches

    # H100 Optimierungen
    tf32=True,                       # H100 spezifisch
    bf16=True,
    fp16=False,

    # Memory Optimierungen
    gradient_checkpointing=True,      # Optional auf H100
    max_grad_norm=0.3,

    # Learning Rate
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,

    # Häufigere Evaluation möglich
    evaluation_strategy="steps",
    eval_steps=50,                   # Häufiger evaluieren
    logging_steps=10,
    save_steps=50,

    # Mehr Checkpoints möglich
    save_total_limit=3,

    # Datenlader
    dataloader_num_workers=4,        # Mehr Worker möglich
    group_by_length=True,            # Bessere Effizienz

    report_to="wandb"
)

Supervised fine-tuning trainer aufsetzen

In [26]:
from transformers.trainer_callback import EarlyStoppingCallback

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # Stoppt wenn 2 Evaluationen keine Verbesserung zeigen
    packing=False
)

NameError: name 'SFTTrainer' is not defined

# Schritt 5 - Modell trainieren

In [ ]:
trainer.train()

wandb.finish()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
trainer.model.push_to_hub(new_model, use_temp_dir=False)